Ingesting Data via API

IMPORTS

In [38]:
# import necessary libraries

import requests
import os
from dotenv import load_dotenv
import pprint
import pandas as pd
import numpy as np

API DECLARATIONS

In [39]:
# Load API key from .env file and make a request to the MBTA API to get all routes

load_dotenv()
url = 'https://api-v3.mbta.com/'
api_key = os.getenv("API_KEY")
headers = {'X-API-Key': api_key}
response = requests.get(f'{url}routes', headers=headers)

RESPONSE AND ERROR CHECKING

In [40]:
# Check if the request was successful and print the data

if response.status_code == 200:
    data = response.json()
    # for route in data['data']:
    #     print(route['attributes']['long_name'], route['attributes']['direction_names'])
    
    df = pd.json_normalize(data['data'])
else:
    print(f'Error: {response.status_code}')

CLEAN UP

In [41]:
# Rename columns to be more descriptive and easier to work with

df = df.rename(columns={
    'attributes.color': 'route_color',
    'attributes.description': 'route_description',
    'attributes.direction_destinations': 'route_destinations',
    'attributes.direction_names': 'route_direction',
    'attributes.fare_class': 'route_fare_class',
    'attributes.listed_route': 'listed_route',
    'attributes.long_name': 'route_long_name', 
    'attributes.short_name': 'route_short_name',
    'attributes.sort_order': 'route_sort_order',
    'attributes.text_color': 'route_text_color',
    'attributes.type': 'route_type',
    'links.self': 'route_link',
    'attributes.id': 'id',
    'relationships.line.data.id': 'line_id' })


# Fill rows with missing values in short_route_name column with EMPTY

df['route_short_name'] = df['route_short_name'].replace('', 'EMPTY')

# Split the route_destinations column into two separate columns for the two directions of the route

df[['start_destination', 'end_destination']] = pd.DataFrame(df['route_destinations'].tolist(), index=df.index)

# Split the route_direction column into two separate columns for the two directions of the route

df[['start_direction', 'end_direction']] = pd.DataFrame(df['route_direction'].tolist(), index=df.index)

# Drop columns that are not needed for the analysis

df = df.drop(columns=[
    'route_type',
    'listed_route',
    'route_text_color',
    'relationships.agency.data.id',
    'relationships.agency.data.type',
    'relationships.line.data.type',
    'route_destinations',
    'route_direction',
    'type'
])

# Reorder columns in alphabetical order

df = df.reindex(columns=[
    'id', 
    'line_id', 
    'route_long_name', 
    'route_short_name', 
    'start_destination', 
    'start_direction', 
    'end_destination', 
    'end_direction', 
    'route_description', 
    'route_fare_class', 
    'route_link', 
    'route_sort_order', 
    'route_color'])


In [42]:
print(df.columns.tolist())

['id', 'line_id', 'route_long_name', 'route_short_name', 'start_destination', 'start_direction', 'end_destination', 'end_direction', 'route_description', 'route_fare_class', 'route_link', 'route_sort_order', 'route_color']


PRINTING + EXPORTS 

In [43]:
# Print the total number of routes and save the cleaned data to a CSV file

print(f"Total routes: {len(data['data'])}")
df.to_csv('mbta_routes.csv', index=False)
print(df)

Total routes: 177
           id        line_id  \
0         Red       line-Red   
1    Mattapan  line-Mattapan   
2      Orange    line-Orange   
3     Green-B     line-Green   
4     Green-C     line-Green   
..        ...            ...   
172       558    line-556558   
173       712    line-712713   
174       713    line-712713   
175       714       line-714   
176       716       line-716   

                                       route_long_name route_short_name  \
0                                             Red Line            EMPTY   
1                                        Mattapan Line            EMPTY   
2                                          Orange Line            EMPTY   
3                                         Green Line B                B   
4                                         Green Line C                C   
..                                                 ...              ...   
172                  Riverside Station - Newton Corner              558 